In [1]:
!pip install transformers[sentencepiece]
!pip install --user -U nltk

In [2]:
import re
import requests
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, logging
import nltk
import torch

Class GetContent for html parsing

In [3]:
from html.parser import HTMLParser

class getContent(HTMLParser):
    def __init__(self, tag:str, attrs:list):
        self.tag = tag
        self.attrs = attrs
        self.intag = 0
        self.result = ""
        HTMLParser.__init__(self)

    def handle_starttag(self, tag, attrs):
        seta1 = set(self.attrs)
        seta2 = set(attrs)
        if tag == self.tag and seta1 == seta2:
            self.intag = 1

    def handle_endtag(self, tag):
        if tag == self.tag and self.intag == 1:
            self.intag = 0

    def handle_data(self, data):
        if self.intag == 1:
            self.result += data + " "


Request data

In [4]:
import urllib3
urllib3.disable_warnings()

userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit"
headers = {'user-agent': userAgent}

Load model

In [5]:

nltk.download('punkt')
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_checkpoint = "ai-forever/RuM2M100-1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint, src_lang="ru", tgt_lang="ru")

model = model.to(device)
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

**Article URL**

In [6]:
articleURL = "https://ria.ru/20240407/pavodki-1938381970.html"

HTML parser

In [7]:
parser = getContent("div", [("class", "article__text")])
htmlResponse = requests.get(articleURL, headers=headers, verify=False)
parser.feed((htmlResponse.content).decode('utf-8'))


Get sentences from the article

In [8]:
sentArticle = nltk.tokenize.sent_tokenize(parser.result, language="russian")


Маскирование токенов в предложении и определение ошибок

In [9]:
def m2m100_sent(text, model, tokenizer):
    specsigns = [('№', 'No')]
    for ss in specsigns:
      text = re.sub(ss[0], ss[1], text)

    # masking whole text and return errors if available
    tr = 0.9 #threshold of the error (parameter !!!)

    p = re.compile(r'[\w-]+')
    listErr = []
    ind = 0

    # собираем данные о словах
    iter = p.finditer(text)
    words = []
    for w in iter:
      words.append([w.group(), w.start(), w.end()])

    inputs = tokenizer(text, text_target=text, return_tensors="pt").to(device)
    logits = model(**inputs).logits
    match_str = ""
    istored = -1

    for i, x in enumerate(inputs["input_ids"][0]):
      if tokenizer.decode(x) not in tokenizer.all_special_tokens:
        match_str += re.escape(tokenizer.decode(x))
        masked_token = x.item()
        mask_token_index = torch.where(inputs["input_ids"] == x)[1]
        mask_token_logits = logits[0, mask_token_index, :]
        probs = torch.nn.functional.softmax(mask_token_logits, dim=1)
        # print(f'Token: [{tokenizer.decode(x)}]; probability: {probs[0][masked_token].item()}')

        # Add to error tokens, if threshold > probability
        if tr > probs[0][masked_token].item():
          merr = re.match(r''+ match_str, text)
          if merr:
            #print (merr, words)
            #print (match_str)
            for w in words:
              if merr.end() > w[1] and merr.end() <= w[2]:
                if istored != i:
                  errorrDesc = {
                    "word": w[0],
                    "start": w[1],
                    "end": w[2],
                    "prob": probs[0][masked_token].item()
                  }
                  listErr.append(errorrDesc)
                  istored = i
          else:
            print (merr, words)
            print (match_str)

        match_str += "\s*"
    return listErr

Перебор всех предложений из статьи и сбор сведений для вывода

In [10]:
outArticle = ""
for sent in sentArticle:
  startPos = 0
  response = m2m100_sent(sent, model, tokenizer)
  if len(response) == 0:
    outArticle += sent + " "
  else:
    for r in response:
      # print (startPos, r)
      outArticle += sent[startPos:r['start']] + "<mark>" + sent[r['start']:r['end']] + "</mark>"
      startPos = r['end']
    outArticle += sent[startPos:] + " "

wrap function

In [11]:
import textwrap
def wrap_text(text, width):
    return textwrap.fill(text, width)

In [12]:
print(wrap_text(outArticle, 80))

САМАРА, <mark>7</mark> <mark>апр</mark> - РИА Новости. Спасатели МЧС
эвакуировали около 820 человек, включая 159 детей, из зоны подтопления в
Самарской области, сообщили журналистам в региональном ГУМЧС России.
"Спасателями МЧС России из зоны подтопления (в Самарской области - ред.) были
эвакуированы 816 человек, в том числе 159 ребенка. В четырех пунктах временного
размещения находятся 62 человека, в том числе 16 детей", - говорится в сообщении
ведомства. Сейчас на территории Самарской области в зоне подтопления находятся
313 жилых домов и 1092 приусадебных участка в Ставропольском,
Большечерниговском, Большеглушицком, Кинель–Черкасском,  Волжском  и других
районах. За сутки водка ушла из 166 домов и 185 приусадебных участков, отметили
в региональном главке МЧС. Помимо эвакуации населения с подтопленных территорий,
спасатели МЧС России доставляют продукты питания.
